In [1]:
import pandas as pd
import numpy as np
# import tensorflow as tf
# import os
# import matplotlib.pyplot as plt
# from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
npInputDataset = np.load(f"./../../Solver/DATA/npInputDataset.npy")
npXTargetDataset= np.load(f"./../../Solver/DATA/npXTargetDataset.npy")
npYTargetDataset= np.load(f"./../../Solver/DATA/npYTargetDataset.npy")
npUTargetDataset= np.load(f"./../../Solver/DATA/npUTargetDataset.npy")
npVTargetDataset= np.load(f"./../../Solver/DATA/npVTargetDataset.npy")
npAccXTargetDataset = np.load(f"./../../Solver/DATA/npXTargetDataset.npy")
npAccYTargetDataset= np.load(f"./../../Solver/DATA/npAccYTargetDataset.npy")
npRhoTargetDataset = np.load(f"./../../Solver/DATA/npRhoTargetDataset.npy")
npPTargetDataset = np.load(f"./../../Solver/DATA/npPTargetDataset.npy")
npCollidingTotal = np.load(f"./../../Solver/DATA/npCollidingTotal.npy")

In [3]:
npXTargetDataset = np.expand_dims(npXTargetDataset, axis=-1)
npYTargetDataset= np.expand_dims(npYTargetDataset, axis=-1)
npUTargetDataset= np.expand_dims(npUTargetDataset, axis=-1)
npVTargetDataset= np.expand_dims(npVTargetDataset, axis=-1)
npAccXTargetDataset = np.expand_dims(npAccXTargetDataset, axis=-1)
npAccYTargetDataset= np.expand_dims(npAccYTargetDataset, axis=-1)
npRhoTargetDataset =np.expand_dims(npRhoTargetDataset, axis=-1)
npPTargetDataset = np.expand_dims(npPTargetDataset, axis=-1)
npCollidingTotal = np.expand_dims(npCollidingTotal,axis=-1)

In [4]:
np.random.RandomState(42)

RandomState(MT19937) at 0x7F42C48EAD40

In [5]:
# The number of data
NData = 100

# The number of particles
NParticles = 100

# The number of TimeSteps
NTimeSteps = 200

# The number of parameters
NParameters = 4

# The number of Targets
NTargets = 8

# Machine Learning

In [6]:
class Neuron:
    def __init__(self, targetTitle):
        
        self.targetTitle = targetTitle
        self.w = None
        self.b = None
        self.losses = np.array([])
        self.val_losses = np.array([])
        self.predict_losses = np.array([])
        self.w_history = None
        self.learningRate = None
        self.timeStep = None
        self.NDatas = None
        self.NParticles = None
        self.NParameters = None
        self.NTargets = None
        self.X_train = None
        self.X_test = None
        self.X_Val = None
        self.y_train = None
        self.y_test = None
        self.y_val = None

    def setup(self, inputDataset, targetDataset, randW, randb,test_size=0.25, random_state=42, learningRate=0.001):
        
        # Variables
        self.timeStep = inputDataset.shape[1]
        self.NDatas = inputDataset.shape[0] 
        self.NParticles = inputDataset.shape[2]
        self.NParameters = inputDataset.shape[3]
        self.NTargets = targetDataset.shape[0]
        self.learningRate = learningRate
        
        self.w = randW
        self.b = randb
        
               
        # Dataset splitting
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(inputDataset, targetDataset,
                                                                                test_size=test_size,
                                                                                random_state=random_state)

        
        
        self.X_train, self.X_Val, self.y_train, self.y_val = train_test_split(self.X_train, self.y_train,
                                                                              test_size=test_size,
                                                                              random_state=random_state)
        
        # Sklearn Input Dataset preprocessing
        # Triainset
        from sklearn.preprocessing import MinMaxScaler
        scaler =MinMaxScaler()
        NTrainData = self.X_train.shape[0]
        self.X_train = self.X_train.reshape(-1,self.NParameters)
        self.X_train = scaler.fit_transform(self.X_train)
        self.X_train = self.X_train.reshape(NTrainData,self.timeStep,self.NParticles,self.NParameters)
        # Testset
        NTestData = self.X_test.shape[0]
        self.X_test = self.X_test.reshape(-1, self.NParameters)
        self.X_test = scaler.transform(self.X_test)
        self.X_test = self.X_test.reshape(NTestData,self.timeStep,self.NParticles,self.NParameters)
        

    def forpass(self, x):
        y_hat = x * self.w + self.b
        return y_hat

    def backprop(self, x, err):
        w_grad = x * err
        b_grad = 1 * err
        return w_grad, b_grad

    def update_val_loss(self):


        y_hat = self.forpass(self.X_Val)
        y_hat = np.sum(y_hat, axis=-1)
        y_hat = np.expand_dims(y_hat, axis=-1)
        err = -(self.y_val - y_hat)
        self.val_losses = np.append(self.val_losses, np.sum(err))

        
    def prediction(self):
        
        NTest = self.X_test.shape[0]
        y_hat = self.forpass(self.X_test)
        y_hat = np.sum(y_hat, axis=-1)
        y_hat = np.expand_dims(y_hat, axis=-1)
        
        self.predict_losses = -(self.y_test - y_hat)
    
    def saveResults(self,address,modelName):
        
        np.save(f"{address}/{modelName}weights.npy",self.w)
        np.save(f"{address}/{modelName}bias.npy",self.b)
        np.save(f"{address}/{modelName}losses.npy",self.losses)
        np.save(f"{address}/{modelName}val_losses.npy",self.val_losses)
        np.save(f"{address}/{modelName}predict_losses.npy",self.predict_losses)
        np.save(f"{address}/{modelName}X_train.npy",self.X_train)
        np.save(f"{address}/{modelName}X_test.npy",self.X_test)
        np.save(f"{address}/{modelName}X_Val.npy",self.X_Val)
        np.save(f"{address}/{modelName}y_train.npy",self.y_train)
        np.save(f"{address}/{modelName}y_test.npy",self.y_test)
        np.save(f"{address}/{modelName}y_val.npy",self.y_val)

    def fit(self, epochs=10):
        
        self.w_history = self.w
        np.random.seed(42)
        indexes = np.random.permutation(np.arange(self.X_train.shape[0]))
        for _ in range(epochs):
            loss = 0
            for i in indexes:                
                y_hat = self.forpass(self.X_train[i])
                y_hat = np.sum(y_hat, axis=-1)
                y_hat = np.expand_dims(y_hat, axis=-1)
                err = -(self.y_train[i] - y_hat)
                w_grad, b_grad = self.backprop(self.X_train[i], err)
                self.w -= w_grad * self.learningRate
                self.b -= b_grad * self.learningRate
                loss += np.sum(err)
            self.losses = np.append(self.losses, loss)
            self.update_val_loss()
        

In [7]:
# Epochs
Epochs = 3000

# stadard deviation of random initiai weights
stdScale = 0.01


# Model 1
Model1Epochs = 3000
Model1LearningRate = 0.001

# Model 2
Model2Epochs = 3000
Model2LearningRate = 0.001

# Model 3
Model3Epochs = 3000
Model3LearningRate = 0.001

# Model 4
Model4Epochs = 3000
Model4LearningRate = 0.001

# Model 5
Model5Epochs = 3000
Model5LearningRate = 0.001

# Model 6
Model6Epochs = 3000
Model6LearningRate = 0.001

# Model 7
Model7Epochs = 3000
Model7LearningRate = 0.001

# Model 8
Model8Epochs = 3000
Model8LearningRate = 0.001

In [8]:
Model1 = Neuron(targetTitle="The x-coordinates of the particles[m]")
Model1.setup(npInputDataset,npXTargetDataset,
             randW = np.random.normal(scale= stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model1LearningRate)

Model1.fit(epochs=Model1Epochs)

In [9]:
Model2 = Neuron(targetTitle="The y-coordinates of the particles[m]")
Model2.setup(npInputDataset,npYTargetDataset,
             randW = np.random.normal(scale=  stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model2LearningRate)

Model2.fit(epochs=Model2Epochs)

In [10]:
Model3 = Neuron(targetTitle="The x-direction velocity of the particles[m/s]")
Model3.setup(npInputDataset,npUTargetDataset,
            randW = np.random.normal(scale=  stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model3LearningRate)
Model3.fit(epochs=Model3Epochs)

In [11]:
Model4 = Neuron(targetTitle="The y-direction velocity of the particles[m/s]")
Model4.setup(npInputDataset,npVTargetDataset,
             randW = np.random.normal(scale= stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model4LearningRate)
Model4.fit(epochs=Model4Epochs)

In [12]:
Model5 = Neuron(targetTitle="The x-direction accelerlation of the particles[m/s^2]")
Model5.setup(npInputDataset,npAccXTargetDataset,
             randW = np.random.normal(scale= stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model5LearningRate)

Model5.fit(epochs=Model5Epochs)

In [13]:
Model6 = Neuron(targetTitle="The y-direction accelerlation of the particles[m/s^2]")
Model6.setup(npInputDataset,npAccYTargetDataset,
              randW = np.random.normal(scale= stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model6LearningRate)
Model6.fit(epochs=Model6Epochs)


In [14]:
Model7 = Neuron(targetTitle="The density of particles[kg/m^3]")
Model7.setup(npInputDataset,npRhoTargetDataset,
              randW = np.random.normal(scale= stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model7LearningRate)

Model7.fit(epochs=Model7Epochs)

In [15]:
Model8 = Neuron(targetTitle="The pressure of particles[Pa]")
Model8.setup(npInputDataset,npPTargetDataset,
               randW = np.random.normal(scale= stdScale ,size=(NTimeSteps,NParticles,NParameters)),
             randb = np.zeros(shape=(NTimeSteps,NParticles,NParameters)),
             learningRate=Model8LearningRate)
Model8.fit(epochs=Model8Epochs)

# Save

#### Epochs: 3000

In [38]:
#
SaveAddress = "./Results"
saveAddressValLosses = "Results_3000epochs_std_set_0.01_valLosses.xlsx"

In [39]:
Model1.prediction()
Model2.prediction()
Model3.prediction()
Model4.prediction()
Model5.prediction()
Model6.prediction()
Model7.prediction()
Model8.prediction()

In [40]:
Model1.saveResults(SaveAddress ,"Model1")
Model2.saveResults(SaveAddress ,"Model2")
Model3.saveResults(SaveAddress ,"Model3")
Model4.saveResults(SaveAddress ,"Model4")
Model5.saveResults(SaveAddress ,"Model5")
Model6.saveResults(SaveAddress ,"Model6")
Model7.saveResults(SaveAddress ,"Model7")
Model8.saveResults(SaveAddress ,"Model8")

In [41]:
dfValLosses = np.abs(np.hstack([Model1.val_losses.reshape(Epochs,-1),Model2.val_losses.reshape(Epochs,-1),
                         Model3.val_losses.reshape(Epochs,-1),Model4.val_losses.reshape(Epochs,-1),
                         Model5.val_losses.reshape(Epochs,-1),Model6.val_losses.reshape(Epochs,-1),
                         Model7.val_losses.reshape(Epochs,-1),Model8.val_losses.reshape(Epochs,-1)]))
dfValLosses = pd.DataFrame(dfValLosses)
dfValLosses.to_excel(f"{SaveAddress}/{saveAddressValLosses}")